In [8]:
from google.cloud import bigquery

client = bigquery.Client(project="valued-lambda-478823-h0")

query = """
SELECT
*
FROM `valued-lambda-478823-h0.azdd_test.age_buckets`
"""


df = client.query(query).to_dataframe()

df.head()

,county,dem_lt18,dem_18_24,dem_25_34,dem_35_44,dem_45_54,dem_55_64,dem_65_74,dem_75,rep_lt18,...,rep_65_74,rep_75,third_lt18,third_18_24,third_25_34,third_35_44,third_45_54,third_55_64,third_65_74,third_75
0,Cheyenne,0,6,17,15,12,34,33,31,10,...,130,93,6,52,94,56,46,51,35,23
1,Hinsdale,0,3,8,7,11,24,28,18,4,...,117,57,0,29,33,30,33,49,36,11
2,Mineral,0,7,33,23,31,66,73,47,0,...,88,61,2,16,45,45,29,41,36,17
3,San Juan,0,3,21,19,31,35,46,22,0,...,41,20,1,17,69,65,69,72,55,28
4,Baca,1,12,34,46,52,108,121,146,14,...,240,222,27,142,226,142,123,140,103,55


In [9]:
id_cols = ["county"]
value_cols = [c for c in df.columns if c not in id_cols]

# unpivot the wide version so looker will play nice
long_df = df.melt(
    id_vars=id_cols,
    value_vars=value_cols,
    var_name="party_age",
    value_name="count"
)

long_df[["party", "age_bucket"]] = long_df["party_age"].str.split("_", n=1, expand=True)
long_df = long_df[id_cols + ["party", "age_bucket", "count"]]

age_sort_map = {
    "lt18": 0,
    "18_24": 1,
    "25_34": 2,
    "35_44": 3,
    "45_54": 4,
    "55_64": 5,
    "65_74": 6,
    "75": 7,
    "75_plus": 7,
}


long_df["sort_key"] = long_df["age_bucket"].map(age_sort_map).fillna(99).astype(int)

long_df.head(10)


,county,party,age_bucket,count,sort_key
0,Cheyenne,dem,lt18,0,0
1,Hinsdale,dem,lt18,0,0
2,Mineral,dem,lt18,0,0
3,San Juan,dem,lt18,0,0
4,Baca,dem,lt18,1,0
5,Dolores,dem,lt18,0,0
6,Jackson,dem,lt18,1,0
7,Kiowa,dem,lt18,0,0
8,Bent,dem,lt18,3,0
9,Custer,dem,lt18,0,0


In [10]:
from google.cloud import bigquery

project = "valued-lambda-478823-h0"
dataset = "azdd_test"
table = f"{project}.{dataset}.party_age_long"

client = bigquery.Client(project=project)

job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE"
)

load_job = client.load_table_from_dataframe(
    long_df,
    destination=table,
    job_config=job_config
)

load_job.result()

print(f"Loaded {client.get_table(table).num_rows} rows into {table}")


Loaded 1536 rows into valued-lambda-478823-h0.azdd_test.party_age_long
